In [19]:
import pandas as pd
import torch
import numpy as np
import torch.utils.data as Data


In [16]:
tisdb=pd.read_csv("/data/reference/human/TIS/human_tisdb_data_1.0.csv")

In [17]:
tisdb.head()

,Gene,Transcript,Chr,Strand,Coordinate_start_ codon,Start_Codon,Coordinate_stop_codon,Stop_Codon,TIS_type,Predicted_uORF
0,RANGRF,NM_001177802,chr17,+,8192082,ATG,8192175,TGA,uTIS,No
1,RANGRF,NM_016492,chr17,+,8192082,ATG,8192175,TGA,uTIS,No
2,RANGRF,NM_001177801,chr17,+,8192082,ATG,8192175,TGA,uTIS,No
3,HIST1H4B,NM_003544,chr6,-,26027228,ATG,26027170,TAA,dTIS,No
4,FGFR1OP2,NM_001171887,chr12,+,27107092,ATG,27117664,TGA,aTIS,No


In [18]:
test=tisdb[tisdb.TIS_type=="uTIS"]

In [19]:
test.head()

,Gene,Transcript,Chr,Strand,Coordinate_start_ codon,Start_Codon,Coordinate_stop_codon,Stop_Codon,TIS_type,Predicted_uORF
0,RANGRF,NM_001177802,chr17,+,8192082,ATG,8192175,TGA,uTIS,No
1,RANGRF,NM_016492,chr17,+,8192082,ATG,8192175,TGA,uTIS,No
2,RANGRF,NM_001177801,chr17,+,8192082,ATG,8192175,TGA,uTIS,No
7,RNF10,NM_014868,chr12,+,120972227,GTG,120972458,TAG,uTIS,Yes
8,RNF10,NM_014868,chr12,+,120972492,ATG,120972501,TGA,uTIS,Yes


In [15]:
atis=open("/data/jeff/work/learning/learning_resource/ATS/atis.input",'r')
atis_all_list=[]
with atis as atis_all:
    for line in atis_all:
        atis_all_list.append(one_hot_encode(line.strip()))
dtis=open("/data/jeff/work/learning/learning_resource/ATS/dtis.input",'r')
dtis_all_list=[]
with dtis as dtis_all:
    for line in dtis_all:
        dtis_all_list.append(one_hot_encode(line.strip()))
utis=open("/data/jeff/work/learning/learning_resource/ATS/utis.input",'r')
utis_all_list=[]
with utis as utis_all:
    for line in utis_all:
        utis_all_list.append(one_hot_encode(line.strip()))

/home/lijiefu/anaconda3/envs/pytorch_learn/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


In [3]:
def one_hot_encode(seq):
#    map = np.asarray([[0, 0, 0, 0],
#                      [1, 0, 0, 0],
#                      [0, 1, 0, 0],
#                      [0, 0, 1, 0],
#                      [0, 0, 0, 1]])
#	numpy version
	map=torch.tensor([[0, 0, 0, 0],
	                  [1, 0, 0, 0],
	                  [0, 1, 0, 0],
	                  [0, 0, 1, 0],
	                  [0, 0, 0, 1]])
	seq = seq.upper().replace('A', '\x01').replace('C', '\x02')
	seq = seq.replace('G', '\x03').replace('T', '\x04').replace('N', '\x00')
	#return map[np.fromstring(seq, np.int8) % 5] numpy version

	map = map[np.fromstring(seq, np.int8)%5]
	return map


In [13]:
atis_all_list=[]

with atis as atis_all:
    for line in atis_all:
        atis_all_list.append(one_hot_encode(line.strip()))
        

/home/lijiefu/anaconda3/envs/pytorch_learn/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


In [18]:
len(atis_all_list)
len(dtis_all_list)
#len(utis_all_list)

1113

In [20]:
 torch.linspace(1, 10, 10)

tensor([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])